In [1]:
!pip install imutils

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import numpy as np
import cv2
import os
from os import listdir
import shutil
import itertools
import imutils
from tqdm import tqdm
from PIL import Image


In [4]:
def crop_brain_contour(image, plot=False):

    # Convert the image to grayscale, and blur it slightly
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.GaussianBlur(gray, (5, 5), 0)

    # Threshold the image, then perform a series of erosions +
    # dilations to remove any small regions of noise
    thresh = cv2.threshold(gray, 45, 255, cv2.THRESH_BINARY)[1]
    thresh = cv2.erode(thresh, None, iterations=2)
    thresh = cv2.dilate(thresh, None, iterations=2)

    # Find contours in thresholded image, then grab the largest one
    cnts = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    cnts = imutils.grab_contours(cnts)
    c = max(cnts, key=cv2.contourArea)


    # Find the extreme points
    extLeft = tuple(c[c[:, :, 0].argmin()][0])
    extRight = tuple(c[c[:, :, 0].argmax()][0])
    extTop = tuple(c[c[:, :, 1].argmin()][0])
    extBot = tuple(c[c[:, :, 1].argmax()][0])

    # crop new image out of the original image using the four extreme points (left, right, top, bottom)
    new_image = image[extTop[1]:extBot[1], extLeft[0]:extRight[0]]

    if plot:
        plt.figure()

        plt.subplot(1, 2, 1)
        plt.imshow(image)

        plt.tick_params(axis='both', which='both',
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Original Image')

        plt.subplot(1, 2, 2)
        plt.imshow(new_image)

        plt.tick_params(axis='both', which='both',
                        top=False, bottom=False, left=False, right=False,
                        labelbottom=False, labeltop=False, labelleft=False, labelright=False)

        plt.title('Cropped Image')

        plt.show()

    return new_image


In [5]:
def save_new_images(x_set, y_set, folder_name):
    i = 0
    for (img, imclass) in zip(x_set, y_set):
        if imclass == 'glioma':
            cv2.imwrite(folder_name+'glioma/'+str(i)+'.png', img)
        elif imclass == 'meningioma':
            cv2.imwrite(folder_name+'meningioma/'+str(i)+'.png', img)

        else:
            cv2.imwrite(folder_name+'pituitary tumor/'+str(i)+'.png', img)
        i += 1

In [6]:
labels = ['glioma','meningioma','pituitary tumor']

X_train = []
y_train = []
X_test = []
y_test = []

print("Train Set")
print("===========")
image_size = 240
for i in labels:
  # path to dataset
    folderPath = os.path.join('/content/drive/MyDrive/BrainTumorProject/split_data_80_20','train',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = crop_brain_contour(img)
        img = cv2.resize(img, (image_size, image_size))##added
        X_train.append(img)
        y_train.append(i)

X_train = np.array(X_train)
y_train = np.array(y_train)

print("Test Set")
print("=========")
for i in labels:
  # path to dataset
    folderPath = os.path.join('/content/drive/MyDrive/BrainTumorProject/split_data_80_20','val',i)
    for j in tqdm(os.listdir(folderPath)):
        img = cv2.imread(os.path.join(folderPath,j))
        img = crop_brain_contour(img)
        img = cv2.resize(img, (image_size, image_size))##added
        X_test.append(img)
        y_test.append(i)

X_test = np.array(X_test)
y_test = np.array(y_test)

Train Set


100%|██████████| 1405/1405 [00:51<00:00, 27.35it/s]


Test Set


100%|██████████| 352/352 [00:12<00:00, 27.55it/s]


In [7]:
# path to save cropped image
save_new_images(X_train, y_train, folder_name='/content/drive/MyDrive/BrainTumorProject/Cropped2/train/')
save_new_images(X_test, y_test, folder_name='/content/drive/MyDrive/BrainTumorProject/Cropped2/val/')

In [ ]:
!ls '/content/drive/MyDrive/BrainTumorProject/Cropped2/train/glioma' | head -10


0.png
1000.png
1001.png
1002.png
1003.png
1004.png
1005.png
1006.png
1007.png
1008.png


In [10]:
import os

train_glioma = len(os.listdir('/content/drive/MyDrive/BrainTumorProject/Cropped2/train/glioma'))
train_meningioma = len(os.listdir('/content/drive/MyDrive/BrainTumorProject/Cropped2/train/meningioma'))
train_pituitary = len(os.listdir('/content/drive/MyDrive/BrainTumorProject/Cropped2/train/pituitary tumor'))

print(f"Train images - Glioma: {train_glioma}, Meningioma: {train_meningioma}, Pituitary: {train_pituitary}")




Train images - Glioma: 1296, Meningioma: 1420, Pituitary: 1405


In [11]:
import os

val_glioma = len(os.listdir('/content/drive/MyDrive/BrainTumorProject/Cropped2/val/glioma'))
val_meningioma = len(os.listdir('/content/drive/MyDrive/BrainTumorProject/Cropped2/val/meningioma'))
val_pituitary = len(os.listdir('/content/drive/MyDrive/BrainTumorProject/Cropped2/val/pituitary tumor'))

print(f"Validation images - Glioma: {val_glioma}, Meningioma: {val_meningioma}, Pituitary: {val_pituitary}")


Validation images - Glioma: 325, Meningioma: 355, Pituitary: 352
